## Mount drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Preprocessing

Si esegue l'unzip del dataset presente su drive, usando '/content/partX' come destinazione

In [ ]:
!unzip "drive/MyDrive/part4.zip" -d "/content/part4"

Si crea tutta la struttura di directory di destinazione

In [ ]:
import os

# src_path è il path del dataset
# dst_path è il path in cui salveremo il dataset processato
src_path = '/content/part4'
dst_path = '/content/part4_proc'
n_dir = 69

def make_dirs():
  directories = os.listdir(src_path)

  for directory in directories:
    os.mkdir(dst_path+'/'+directory+'_0')
    os.mkdir(dst_path+'/'+directory+'_1')
    
    src_dir_path = src_path+'/'+directory
    files = os.listdir(src_dir_path)
    for file in files:

      if '.avi' in file:
        dst_dir_path = dst_path+'/'+directory+'_0'
        file_no_ext = file[:-4]
        os.mkdir(dst_dir_path+'/'+file_no_ext)

      else:
        dst_dir_path = dst_path+'/'+directory+'_1'
        os.mkdir(dst_dir_path+'/'+file)


def check():
  directories = os.listdir(dst_path)

  if len(directories) is not n_dir*2:
    print("missing directories")
  
  for directory in directories:
    files = os.listdir(dst_path+'/'+directory)
    
    if len(files) is not 5:
      print("missing files")


make_dirs()
check()

Si processa il dataset eseguendo la detection del volto ed il cropping della regione perioculare.<br>
Le sequenze di frame senza mascherina vengono salvate nelle directory xxx_0.<br>
Le sequenze di frame con mascherina vengono salvate nelle directory xxx_1

In [ ]:
import cv2
import dlib
import os

# src_path è il path del dataset
# dst_path è il path in cui salveremo il dataset processato
src_path = '/content/part4'
dst_path = '/content/part4_proc'

# detector e predictor per eseguire la detection del volto
# e stabilire i landmark facciali
detector_dat = 'drive/MyDrive/mmod_human_face_detector.dat'
predictor_dat = 'drive/MyDrive/shape_predictor_68_face_landmarks.dat'
detector = dlib.cnn_face_detection_model_v1(detector_dat)
predictor = dlib.shape_predictor(predictor_dat)

# width ed height sono lunghezza ed altezza dei frame
# dopo essere stati processati
width = 200
height = 200


# per trovare il riferimento (landmark) più in alto
def find_highest_ref(landmarks):
    references = []
    references.append(landmarks.part(17).y)
    references.append(landmarks.part(18).y)
    references.append(landmarks.part(19).y)
    references.append(landmarks.part(20).y)
    references.append(landmarks.part(21).y)
    references.append(landmarks.part(22).y)
    references.append(landmarks.part(23).y)
    references.append(landmarks.part(24).y)
    references.append(landmarks.part(25).y)
    references.append(landmarks.part(26).y)
    return min(references)


# per trovare il riferimento (landmark) più in basso
def find_lowest_ref(landmarks):
    references = []
    references.append(landmarks.part(28).y)
    references.append(landmarks.part(36).y)
    references.append(landmarks.part(41).y)
    references.append(landmarks.part(40).y)
    references.append(landmarks.part(39).y)
    references.append(landmarks.part(42).y)
    references.append(landmarks.part(47).y)
    references.append(landmarks.part(46).y)
    references.append(landmarks.part(45).y)
    return max(references)


# prende in input un frame e dà in output un frame 
# della regione perioculare con size widthxheight 
def crop_frame(frame):
  gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
  faces = detector(gray, 1)

  # se non sono stati trovati volti, restituisci None
  if not faces:
    return None

  # trova i landmark ed esegui il cropping
  for face in faces:
    landmarks = predictor(gray, face.rect)

    # fisso i riferimenti da usare
    left = landmarks.part(0).x
    top = find_highest_ref(landmarks) - 15
    right = landmarks.part(16).x
    bottom = find_lowest_ref(landmarks) + 15

    # taglio la ROI della pericoulare e la riadatto con shape widthxheigh
    frame = frame[top:bottom, left:right]

    # se il frame non è vuoto, faccio la resize
    if frame.size:
      frame = cv2.resize(frame, (width, height))
      return frame

    # altrimenti resituisco None
    else:
      return None


# esegue il cropping della regione perioculare
# sui primi 50 frame di un video
def manage_video(src_dir_path, dst_dir_path, file):
  file_no_ext = file[:-4]
  cap = cv2.VideoCapture(src_dir_path+'/'+file)
  counter = 0

  while counter < 50 and cap.isOpened():
    ret, frame = cap.read()

    if ret:
      frame = crop_frame(frame)

      # salvo il frame croppato
      if frame is not None:
        dst_file_path = dst_dir_path+'/'+file_no_ext
        framename = file_no_ext+'_'+str(counter).zfill(2)+'.jpeg'
        cv2.imwrite(dst_file_path+'/'+framename, frame)
        counter += 1
    else:
      break

  # cleanup
  cap.release()


def manage_file(src_dir_path, dst_dir_path, file):
  src_file_path = src_dir_path+'/'+file
  dst_file_path = dst_dir_path+'/'+file
  framenames = os.listdir(src_file_path)

  for framename in framenames:
    frame = cv2.imread(src_file_path+'/'+framename, 1)
    frame = crop_frame(frame)

    # salvo il frame croppato
    if frame is not None:
      cv2.imwrite(dst_file_path+'/'+framename, frame)


def main():
  directories = os.listdir(src_path)

  for directory in directories:
    src_dir_path = src_path+'/'+directory
    files = os.listdir(src_dir_path)

    for file in files:
      print("computing file %s/%s" % (src_dir_path, file))

      if '.avi' in file:
        dst_dir_path = dst_path+'/'+directory+'_0'
        manage_video(src_dir_path, dst_dir_path, file)

      else:
        dst_dir_path = dst_path+'/'+directory+'_1'
        manage_file(src_dir_path, dst_dir_path, file)


main()

Si zippa il dataset processato

In [ ]:
!zip -r part4_proc.zip part4_proc

## Preprocessing video non conformi

In [ ]:
!unzip "drive/MyDrive/XM2VTS.zip" -d "XM2VTS"
!unzip "drive/MyDrive/XMY2VTS.zip" -d "XMY2VTS"

Streaming output truncated to the last 5000 lines.
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_16.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_17.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_18.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_19.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_20.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_21.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_22.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_23.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_24.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_25.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_26.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_27.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_28.jpeg  
  inflating: XMY2VTS/XMY2VTS/236_1/281_3_4to5/281_3_4to5_29.jpeg  
  inflating

In [ ]:
import cv2
import dlib
import os

# src_path è il path del dataset
# dst_path è il path in cui salveremo il dataset processato
src_path = '/content/XM2VTS/XM2VTS'
dst_path = '/content/XMY2VTS/XMY2VTS'

# detector e predictor per eseguire la detection del volto
# e stabilire i landmark facciali
detector_dat = 'drive/MyDrive/mmod_human_face_detector.dat'
predictor_dat = 'drive/MyDrive/shape_predictor_68_face_landmarks.dat'
detector = dlib.cnn_face_detection_model_v1(detector_dat)
predictor = dlib.shape_predictor(predictor_dat)

# width ed height sono lunghezza ed altezza dei frame
# dopo essere stati processati
width = 200
height = 200


# per trovare il riferimento (landmark) più in alto
def find_highest_ref(landmarks):
    references = []
    references.append(landmarks.part(17).y)
    references.append(landmarks.part(18).y)
    references.append(landmarks.part(19).y)
    references.append(landmarks.part(20).y)
    references.append(landmarks.part(21).y)
    references.append(landmarks.part(22).y)
    references.append(landmarks.part(23).y)
    references.append(landmarks.part(24).y)
    references.append(landmarks.part(25).y)
    references.append(landmarks.part(26).y)
    return min(references)


# per trovare il riferimento (landmark) più in basso
def find_lowest_ref(landmarks):
    references = []
    references.append(landmarks.part(28).y)
    references.append(landmarks.part(36).y)
    references.append(landmarks.part(41).y)
    references.append(landmarks.part(40).y)
    references.append(landmarks.part(39).y)
    references.append(landmarks.part(42).y)
    references.append(landmarks.part(47).y)
    references.append(landmarks.part(46).y)
    references.append(landmarks.part(45).y)
    return max(references)


# esegue il cropping della regione perioculare
# sui primi 50 frame di un video
def manage_video(src_dir_path, dst_dir_path, file):
  file_no_ext = file[:-4]
  cap = cv2.VideoCapture(src_dir_path+'/'+file)
  counter = 0

  while counter < 50 and cap.isOpened():
    ret, frame = cap.read()

    if ret:
      gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
      faces = detector(gray, 1)

      # trova i landmark ed esegui il cropping
      for face in faces:
        landmarks = predictor(gray, face.rect)

        # fisso i riferimenti da usare
        left = landmarks.part(0).x
        top = find_highest_ref(landmarks) - 15
        right = landmarks.part(16).x
        bottom = find_lowest_ref(landmarks) + 15

        # taglio la ROI della pericoulare e la riadatto con shape widthxheigh
        # sul frame con mascherina
        dst_file_path = dst_dir_path+'/'+file_no_ext
        framename = file_no_ext+'_'+str(counter).zfill(2)+'.jpeg'
        frame = cv2.imread(dst_file_path+'/'+framename, 1)
        frame = frame[top:bottom, left:right]

        # se il frame non è vuoto, faccio la resize e lo salvo
        if frame.size:
          frame = cv2.resize(frame, (width, height))
          cv2.imwrite(dst_file_path+'/'+framename, frame)
          counter += 1

  # cleanup
  cap.release()


def main():
  directories = os.listdir(src_path)

  for directory in directories:
    src_dir_path = src_path+'/'+directory
    dst_dir_path = dst_path+'/'+directory+'_1'
    files = os.listdir(src_dir_path)

    for file in files:
      print("computing file %s/%s" % (src_dir_path, file))
      manage_video(src_dir_path, dst_dir_path, file)


main()

computing file /content/XM2VTS/XM2VTS/061/065_4_1to2.avi
computing file /content/XM2VTS/XM2VTS/061/065_3_3.avi
computing file /content/XM2VTS/XM2VTS/061/065_4_3.avi
computing file /content/XM2VTS/XM2VTS/061/065_3_4to5.avi
computing file /content/XM2VTS/XM2VTS/061/065_3_1to2.avi
computing file /content/XM2VTS/XM2VTS/062/066_3_4to5.avi
computing file /content/XM2VTS/XM2VTS/062/066_3_3.avi
computing file /content/XM2VTS/XM2VTS/062/066_4_1to2.avi
computing file /content/XM2VTS/XM2VTS/062/066_3_1to2.avi
computing file /content/XM2VTS/XM2VTS/062/066_4_3.avi
computing file /content/XM2VTS/XM2VTS/057/060_3_3.avi
computing file /content/XM2VTS/XM2VTS/057/060_4_3.avi
computing file /content/XM2VTS/XM2VTS/057/060_3_1to2.avi
computing file /content/XM2VTS/XM2VTS/057/060_4_1to2.avi
computing file /content/XM2VTS/XM2VTS/057/060_3_4to5.avi
computing file /content/XM2VTS/XM2VTS/044/047_3_1to2.avi
computing file /content/XM2VTS/XM2VTS/044/047_3_4to5.avi
computing file /content/XM2VTS/XM2VTS/044/047_3_3

In [ ]:
!zip -r XMY2VTS_proc.zip XMY2VTS

Streaming output truncated to the last 5000 lines.
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_17.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_30.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_38.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_20.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_08.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_23.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_01.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_19.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_42.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_49.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_36.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_27.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3/130_3_3_43.jpeg (deflated 1%)
  adding: XMY2VTS/XMY2VTS/117_1/130_3_3